In [1]:

"""
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
EXERCICE : ACCELERATION GPU D'UN CNN (basique) CODE EN PYTHON/NUMPY

 -> Le but de cet exercice est
    (1) de bien comprendre comment les parametres d'un reseau de neurones sont appris
     et en particulier comment fonctionne la backpropagation. L'implementation  python
    de differentes couches de CNN est donnee. Le CNN apprend alors a reconnaitre des 0
    et des 1 issus du jeu de donnees MNIST.
    (2) rendre le code le plus rapide possible en utilisant de l'acceleration GPU avec
    CUDA.

-> Une fois le code compris ... un seul objectif : l'accelerer avec du code GPU !

-> Conseil : Focalisez vous sur l'amélioration de 'conv_backward' qui est appelée
              dans la cellule commencant par TRAINING dans MAIN. Vous verrez qu'une
              version en Python brut y est actuellement appelée et qu'une version
              en CUDA non-parallélisée peut y etre utilisée. C'est cette fonction que
              nous allons chercher à améliorer.
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
"""


import numpy as np
import matplotlib.pyplot as plt
from time import time


In [2]:
#CUDA STUFFS

#!pip install pycuda

import pycuda.driver as cuda
from pycuda import driver, compiler, gpuarray, tools 
import pycuda.autoinit


MyDevice=pycuda.driver.Device(0)
#MyDevice=pycuda.driver.Device()
MyDevice.get_attributes()

{pycuda._driver.device_attribute.ASYNC_ENGINE_COUNT: 1,
 pycuda._driver.device_attribute.CAN_MAP_HOST_MEMORY: 1,
 pycuda._driver.device_attribute.CLOCK_RATE: 1137000,
 pycuda._driver.device_attribute.COMPUTE_CAPABILITY_MAJOR: 5,
 pycuda._driver.device_attribute.COMPUTE_CAPABILITY_MINOR: 0,
 pycuda._driver.device_attribute.COMPUTE_MODE: pycuda._driver.compute_mode.DEFAULT,
 pycuda._driver.device_attribute.CONCURRENT_KERNELS: 1,
 pycuda._driver.device_attribute.ECC_ENABLED: 0,
 pycuda._driver.device_attribute.GLOBAL_L1_CACHE_SUPPORTED: 0,
 pycuda._driver.device_attribute.GLOBAL_MEMORY_BUS_WIDTH: 128,
 pycuda._driver.device_attribute.GPU_OVERLAP: 1,
 pycuda._driver.device_attribute.INTEGRATED: 0,
 pycuda._driver.device_attribute.KERNEL_EXEC_TIMEOUT: 1,
 pycuda._driver.device_attribute.L2_CACHE_SIZE: 2097152,
 pycuda._driver.device_attribute.LOCAL_L1_CACHE_SUPPORTED: 1,
 pycuda._driver.device_attribute.MANAGED_MEMORY: 1,
 pycuda._driver.device_attribute.MAXIMUM_SURFACE1D_LAYERED_LAYERS: 20

In [3]:

#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#                     dense layer
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

def init_dense_layer_weights(input_size, output_size):
  """
  INSPIRED BY:  https://towardsdatascience.com/building-neural-network-from-scratch-9c88535bf8e9

  initialize the weights of a dense layer
  """
  weights = np.random.normal(loc=0.0,scale = np.sqrt(2/(input_size+output_size)),
                                        size = (input_size,output_size))
  return weights

def dense_forward(input,weights):
  """
  INSPIRED BY:  https://towardsdatascience.com/building-neural-network-from-scratch-9c88535bf8e9

  forward phase of a dense layer
  """
  cache = (input,weights)
  output = np.dot(input,weights)

  return output, cache

def dense_backward(grad_output,cache):
  """
  INSPIRED BY:  https://towardsdatascience.com/building-neural-network-from-scratch-9c88535bf8e9

  backward phase of a dense layer. The cache was returned by dense_forward.
  """
  input,weights = cache

  grad_input = np.dot(grad_output, weights.T)

  grad_weights = np.dot(input.T, grad_output)

  return grad_input , grad_weights


#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#                     ReLU layer
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

def ReLU_forward(input):
  """
  INSPIRED BY:  https://towardsdatascience.com/building-neural-network-from-scratch-9c88535bf8e9
  """

  cache = (input)
  output = np.maximum(0,input)
  return output , cache

def ReLU_backward(grad_output,cache):
  """
  INSPIRED BY:  https://towardsdatascience.com/building-neural-network-from-scratch-9c88535bf8e9
  """

  input = cache
  relu_grad = input > 0
  grad_input=grad_output*relu_grad

  return grad_input

#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#                     logistic function
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

def Logistic_function(input):
  return 1 / (1 + np.exp(-input))

def derivative_Logistic_function(input):
  lf_input=Logistic_function(input)
  return lf_input*(1-lf_input)



In [4]:

#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#                     convolutional layer - CPU
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


def init_convolution_filter(size):
  """
  size must be a list of the shape [size_h,size_w]
  """
  w=(0.5+np.random.randn(size[0],size[1]))/(size[0]*size[1])
  return w


def conv_forward(input, w):
    """
    INSPIRED BY: https://gist.github.com/neodelphis
    Remark: here stride=1 / no padding / image has only one layer / only one filter

    A naive implementation of the forward pass for a convolutional layer.
    The input consists of N observations, each of height H and
    width W.
    We convolve each input with a filter of height HH and width WW.
    Input:
    - input: Input data of shape (N, H, W)
    - w: Filter weights of shape (HH, WW)

    Returns a tuple of:
    - output: Output data, of shape (N, H', W') where H' and W' are given by H'=H and W'=W
    - cache: (input, w)
    """

    N, H, W = input.shape
    HH, WW = w.shape

    # dimensions de la sortie (tests sur la validité des choix necessaires ensuite / sinon padding)
    H_ = H
    W_ = W

    output = np.zeros((N, H_, W_))

    # Version sans vectorisation
    for n in range(N):       # On parcourt toutes les images
            for i in range(H_): # indices du résultat
                for j in range(W_):
                    for k in range(HH): # indices du filtre
                        for l in range(WW):
                          if i+k<H and j+l<W:   #test whether we're inside the image
                                output[n,i,j] += input[n, i+k, j+l] * w[k, l]

    cache = (input, w)
    return output, cache



def conv_backward(grad_output, cache):
    """
    INSPIRED BY: https://gist.github.com/neodelphis
    Remark: here stride=1 / no padding / image has only one layer / only one filter

    A naive implementation of the backward pass for a convolutional layer.

    Inputs:
    - grad_output: Upstream derivatives. -> Gradient of the loss at the output of the layer
    - cache: A tuple of (input, w) as in conv_forward_naive

    Returns a tuple of:
    - grad_input: Gradient of the loss at the input of the layer wrt input
    - grad_w: Gradient of the loss at the input of the layer wrt w
    """

    # Récupération des variables
    input, w = cache

    # Initialisations
    grad_input = np.zeros_like(input)
    grad_w = np.zeros_like(w)

    # Dimensions
    N, H, W = input.shape
    HH, WW = w.shape
    _, H_, W_ = grad_output.shape   #H_ and W_ sould be equal to H and W

    # Version sans vectorisation
    for n in range(N):       # On parcourt toutes les images
            for i in range(HH): # indices du résultat
                for j in range(WW):
                    for k in range(H_): # indices du filtre
                        for l in range(W_):
                          if i+k<H_ and j+l<W_:
                                grad_w[i,j] += input[n, i+k, j+l] * grad_output[n, k, l]


    # Version sans vectorisation
    for n in range(N):       # On parcourt toutes les images
            for i in range(H): # indices de l'entrée participant au résultat
                for j in range(W):
                    for k in range(HH): # indices du filtre
                        for l in range(WW):
                          if i+k<H and j+l<W:
                                grad_input[n,i,j] += grad_output[n, i+k, j+l] * w[HH-k-1,WW-l-1]

    return grad_input, grad_w



In [5]:
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#                     convolutional layer - CUDA-GPU VERSION 
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

kernel_code_tpl = """

__global__ void cpt_grad_w_Kernel_sequential(float *input, float *grad_output, float *grad_w)
{
    int bidon = threadIdx.x;
    
    float tmp;
    int i,j,k,l,n;
    const int nb_pixels_in_image = %(IMAGE_SIZE)s * %(IMAGE_SIZE)s;


    for (i=0;i<%(CONV_KERNEL_SIZE)s;i++){
        for (j=0;j<%(CONV_KERNEL_SIZE)s;j++){
            grad_w[j* %(CONV_KERNEL_SIZE)s + i]=0.;
            for (n=0;n<%(BATCH_SIZE)s;n++){
                tmp=0.;
                for (k=0;k<%(IMAGE_SIZE)s-i;k++){      
                    for (l=0;l<%(IMAGE_SIZE)s-j;l++){
                        if ((i+k<%(IMAGE_SIZE)s) && (j+l<%(IMAGE_SIZE)s)){
                            tmp+=input[n*nb_pixels_in_image + (i+k)* %(IMAGE_SIZE)s + (j+l)]*grad_output[n*nb_pixels_in_image + k* %(IMAGE_SIZE)s + l];
                        }
                    }
                }
                grad_w[j* %(CONV_KERNEL_SIZE)s + i]+=tmp;
            }
        }
    }
}


__global__ void cpt_grad_input_Kernel_sequential(float *grad_output, float *w, float *grad_input)
{
    int bidon = threadIdx.x;
    float tmp;
    int i,j,k,l,n;
    const int nb_pixels_in_image = %(IMAGE_SIZE)s * %(IMAGE_SIZE)s;

    for (i=0;i<%(IMAGE_SIZE)s;i++){
      for (j=0;j<%(IMAGE_SIZE)s;j++){
        for (n=0;n<%(BATCH_SIZE)s;n++){
            grad_input[n*nb_pixels_in_image + j* %(IMAGE_SIZE)s + i]=0.;
            tmp=0.;
            for (k=0;k<%(CONV_KERNEL_SIZE)s;k++){
              for (l=0;l<%(CONV_KERNEL_SIZE)s;l++){
                if ((i+k<%(IMAGE_SIZE)s)&&(j+l<%(IMAGE_SIZE)s)){
                  tmp+=grad_output[n*nb_pixels_in_image +(i+k)* %(IMAGE_SIZE)s + (j+l)] * w[(%(CONV_KERNEL_SIZE)s-k-1 )* %(CONV_KERNEL_SIZE)s + (%(CONV_KERNEL_SIZE)s-l-1)];
                }
              }
            }
            grad_input[n*nb_pixels_in_image + j* %(IMAGE_SIZE)s + i]+=tmp;
        }
      }
    }
}

"""





#parameters that will be used in the kernels. Must be coherent with what will be used later. 
BATCH_SIZE = 100
IMAGE_SIZE = 28  #image is considered to have IMAGE_SIZE x IMAGE_SIZE pixels
CONV_KERNEL_SIZE = 5  #the convolution kernel is considered to have CONV_KERNEL_SIZE x CONV_KERNEL_SIZE pixels

kernel_code = kernel_code_tpl % {
        'BATCH_SIZE': BATCH_SIZE,
        'IMAGE_SIZE': IMAGE_SIZE,
        'CONV_KERNEL_SIZE': CONV_KERNEL_SIZE,
        }

mod = compiler.SourceModule(kernel_code)

cpt_grad_w_gpu_sequential = mod.get_function("cpt_grad_w_Kernel_sequential")           #sequential version - TO UPDATE WITH PARALLELIZED VERSION
cpt_grad_input_gpu_sequential = mod.get_function("cpt_grad_input_Kernel_sequential")   #sequential version - TO UPDATE WITH PARALLELIZED VERSION


def conv_backward_GPU(grad_output, cache):
    """
    Same as conv_backward with GPU code 
    """

    # Récupération des variables
    input, w = cache

    # Initialisations
    grad_input = np.zeros_like(input).astype(np.float32)
    grad_w = np.zeros_like(w).astype(np.float32)

    # Dimensions
    N, H, W = input.shape
    HH, WW = w.shape
    _, H_, W_ = grad_output.shape   #H_ and W_ sould be equal to H and W

    # Version GPU
    w_gpu = gpuarray.to_gpu(w)

    grad_w_gpu = gpuarray.to_gpu(grad_w)


    #copy gradient related information from the CPU to the GPU 
    input_gpu = gpuarray.to_gpu(input[:,:,:].astype(np.float32))
    grad_output_gpu = gpuarray.to_gpu(grad_output[:,:,:].astype(np.float32))
    grad_input_gpu = gpuarray.to_gpu(grad_input[:,:,:])
            
    #gradient estimations in the GPU 
    cpt_grad_w_gpu_sequential(input_gpu, grad_output_gpu, grad_w_gpu , block=(1,1,1))     #sequential version - TO UPDATE WITH PARALLELIZED VERSION
    cpt_grad_input_gpu_sequential(grad_output_gpu, w_gpu, grad_input_gpu, block=(1,1,1))  #sequential version - TO UPDATE WITH PARALLELIZED VERSION
      
    #copy the gradients from the GPU to the CPU 
    grad_w=grad_w_gpu.get()
    grad_input[:,:,:]=grad_input_gpu.get()

    return grad_input, grad_w



In [6]:

#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#                     MAIN
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

#1) INIT

#get and treat data
data=np.genfromtxt('./mnist_0_1.csv',delimiter=',')

n_tot=data.shape[0]
p=data.shape[1]

y_train=data[:int(2.*n_tot/3.),0].reshape(-1,1)
X_train=(data[:int(2.*n_tot/3.),1:]/(255.*p)).reshape(-1,28,28)

y_test=data[int(2.*n_tot/3.):,0].reshape(-1,1)
X_test=(data[int(2.*n_tot/3.):,1:]/(255.*p)).reshape(-1,28,28)


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)




def ShowMNISTObservation(X_data,y_data,obsNb=0):
  plt.clf()
  plt.imshow(X_data[obsNb,:].reshape((28,28)))
  plt.title('Observation '+str(obsNb)+': Label '+str((y_data[obsNb,0])))
  plt.show()



#ShowMNISTObservation(X_train,y_train,0)
#ShowMNISTObservation(X_train,y_train,1)
#...
#ShowMNISTObservation(X_test,y_train,1)



(2000, 28, 28)
(2000, 1)
(1000, 28, 28)
(1000, 1)


In [ ]:

#2) TRAINING


#generate the network parameters
w_conv=init_convolution_filter([5,5])
w_dense=init_dense_layer_weights(28*28, 1)

w_conv_init=w_conv.copy()
w_dense_init=w_dense.copy()

#Stochastic gradient Descent
Batch_size=100
nb_epochs=2

n=X_train.shape[0]

total_time_in_conv_backward=0.
start_time= time()

for epoch in range(nb_epochs):
    print('epoch:',epoch)
    obsIDs=np.arange(n)
    np.random.shuffle(obsIDs)
    batch_start=0

    while batch_start+Batch_size<n:
      x_batch=X_train[obsIDs[batch_start:batch_start+Batch_size],:,:]
      y_true_batch=y_train[obsIDs[batch_start:batch_start+Batch_size],:]

      #forward phase
      output_1 , cache_1 = conv_forward(x_batch, w_conv)
      output_2 , cache_2 = ReLU_forward(output_1)
      output_3 , cache_3 = dense_forward(output_2.reshape(-1,28*28),w_dense) #the reshape is used to flatten the image
      y_pred = Logistic_function(output_3)

      MSE_loss = np.mean( np.power(y_pred-y_true_batch,2.) ) #log-likelihood would be slightly more general

      #backward phase
      grad_MSE_loss = 2*(y_pred-y_true_batch)
      grad_output_3=grad_MSE_loss*derivative_Logistic_function(output_3)
      grad_output_2 , grad_w_dense = dense_backward(grad_output_3,cache_3)
      grad_output_1=ReLU_backward(grad_output_2.reshape(-1,28,28),cache_2)  #the reshape reverts the flattening
      
      rtime = time()
      grad_input, grad_w_conv=conv_backward(grad_output_1, cache_1)       #!!! VERSION PYTHON BRUT POUR COMPARAISON
      #grad_input, grad_w_conv=conv_backward_GPU(grad_output_1, cache_1)  #!!! VERSION GPU A AMELIORER
      rtime = time() - rtime
      
      total_time_in_conv_backward+=rtime

      #gradient descent-update
      w_dense-=500*grad_w_dense
      w_conv-=0.001*grad_w_conv    #REMARK: the learning rate has to be much smaller on the convolutional layer than on the dense layer (the data should be re-scaled in this layer to avoid this)

      #prepare the next mini-batch
      batch_start+=Batch_size

      print(MSE_loss)

print('Total time:',time() - start_time)
print('Time in conv_backward:',total_time_in_conv_backward)



epoch: 0
0.24999577594923486
0.23549669298790807
0.21802867055189426
0.19696844459809307
0.17526718744359956
0.1452033075749793
0.14038363528670922
0.09376547887501674
0.07626375067750728
0.05558239305440977
0.04631239312291555
0.03191628393171164
0.042821020530403625
0.026237674228690448
0.023124438988854693
0.026927049012712825
0.025459176013507046
0.018941295929187866
0.015381000665880228
epoch: 1
0.019326518964739612
0.010851377413260168
0.010427209071109698
0.015272178130120818
0.019172012384682448
0.01056187328601837
0.006213559898535801
0.011049196210636388
0.007930702019579848
0.011824746676024862
0.006037785422248024
0.011348299462663794
0.013616996208325613
0.007023476773415392
0.007871707112799666
0.009093829578459282
0.012882421477064497


**Temps pour conv_backward sur mon laptop (batch size=100 / 2 epochs)** : CPU-python : 111 secondes / GPU séquentiel : 9.8 secondes / GPU parallélisé : *... à vous de trouver ...*  

In [ ]:

#3) TEST

#show the trained information

plt.imshow(w_dense_init.reshape((28,28)))
plt.title('initial dense layer')
plt.show()

plt.imshow(w_dense.reshape((28,28)))
plt.title('Trained dense layer')
plt.show()

print('Initial convolution filter:\n',w_conv_init)
print('Trained convolution filter:\n',w_conv)


#predictions on test data

output_1 , cache_1 = conv_forward(X_test, w_conv)
output_2 , cache_2 = ReLU_forward(output_1)
output_3 , cache_3 = dense_forward(output_2.reshape(-1,28*28),w_dense)
y_pred = Logistic_function(output_3)

MSE_loss = np.mean( np.power(y_pred-y_test,2.) )

Nb_false_pred=np.sum(np.abs(1*(y_pred>0.5)-y_test))

prct_false_pred=100.*Nb_false_pred/y_test.shape[0]

print('Percentage of good predictions:',100-prct_false_pred)
